In [1]:
import pandas as pd, torch
from datasets import load_dataset
from torchtuning import koalpaca_dataset, kowiki_dataset, custom_chat_dataset, custom_instruction_dataset
from torchtune.models.llama3 import llama3_tokenizer
import numpy as np, json

In [2]:
tokenizer = llama3_tokenizer('checkpoints/Meta-Llama-3-8B-Instruct/original/tokenizer.model')

In [6]:
ds = custom_instruction_dataset(
    tokenizer=tokenizer,
    source='csv',
    max_seq_len=1536,
    split='train',
    column_map={'input': 'user', 'output': 'assistant'},
    data_files='data/inst_data_en_ko.csv'
)

In [7]:
ds._data[1]

{'Unnamed: 0': 1,
 'user': '한국어를 배우는 가장 좋은 방법은 뭐야?',
 'assistant': '한국어를 배우는 가장 좋은 방법은 개인의 학습 스타일, 목표, 그리고 사용 가능한 자료에 따라 다르기 때문에 단 하나의 정답은 없습니다. 하지만, 다양한 방법을 결합한 포괄적인 접근 방식을 소개해 드릴게요.\n\n1. **기본부터 시작하기:**\n    - 먼저 한국어 알파벳 (한글)을 배우세요. 한글은 비교적 간단하고 발음대로 적혀 있어서 읽고 쓰는 것을 쉽게 시작할 수 있습니다. 한글 학습을 위한 앱, 유튜브 동영상, 웹사이트 등 온라인 자료가 많이 있으니 활용해보세요.\n    - 기본 한국어 문법을 익히세요. 문장 구조, 동사 활용, 흔히 쓰이는 조사 등을 포함합니다.\n\n2. **몰입하기:**\n    - **언어 교환:** 한국어를 사용하는 사람과 함께 연습할 기회를 찾으세요. 언어 교환 플랫폼을 통해 온라인으로 또는 만남을 통해 오프라인으로 할 수 있습니다. 원어민과 직접 대화하는 것은 발음과 유창성 향상에 큰 도움이 됩니다.\n    - **한국어 미디어:** 한국 드라마, 영화, 예능 프로그램을 시청하세요. 구조적인 학습 방법은 아니지만, 한국어에 익숙해지고 새로운 단어를 배우는 데 도움이 됩니다.\n    - **한국 음악:** K-팝과 다른 한국 음악을 들어보세요. 가사에 주의 깊게 귀 기울이고 의미를 파악하려고 노력하세요.\n\n3. **학습 자료 활용하기:**\n    - **교재와 연습 문제집:** 문법 설명, 단어 목록, 연습 문제가 포함된 체계적인 학습 자료를 제공합니다.\n    - **언어 학습 앱:** 듀오링고, 멤리즈, 로제타 스톤과 같은 앱은 인터랙티브한 수업과 게임을 통해 단어, 문법, 발음을 배우는 데 도움이 됩니다.\n    - **플래시 카드:** 단어와 문법 개념을 암기하기 위해 플래시 카드를 만들거나 구매하세요.\n\n4. **꾸준함이 중요합니다:**\n    - 한국어 학습을 위해 매일 일

In [8]:
print(tokenizer.decode(ds[1][0]))

<|start_header_id|>user<|end_header_id|>

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a assistant is named 'Arch AI' and developed by (주)스마트소셜(SmartSocial, Inc in English).
You have to do your best to help `user` who is chatting with you.
Try to answer in the language the user asked the question.


### Input:
한국어를 배우는 가장 좋은 방법은 뭐야?

### Response:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

한국어를 배우는 가장 좋은 방법은 개인의 학습 스타일, 목표, 그리고 사용 가능한 자료에 따라 다르기 때문에 단 하나의 정답은 없습니다. 하지만, 다양한 방법을 결합한 포괄적인 접근 방식을 소개해 드릴게요.

1. **기본부터 시작하기:**
    - 먼저 한국어 알파벳 (한글)을 배우세요. 한글은 비교적 간단하고 발음대로 적혀 있어서 읽고 쓰는 것을 쉽게 시작할 수 있습니다. 한글 학습을 위한 앱, 유튜브 동영상, 웹사이트 등 온라인 자료가 많이 있으니 활용해보세요.
    - 기본 한국어 문법을 익히세요. 문장 구조, 동사 활용, 흔히 쓰이는 조사 등을 포함합니다.

2. **몰입하기:**
    - **언어 교환:** 한국어를 사용하는 사람과 함께 연습할 기회를 찾으세요. 언어 교환 플랫폼을 통해 온라인으로 또는 만남을 통해 오프라인으로 할 수 있습니다. 원어민과 직접 대화하는 것은 발음과 유창성 